# Instrukcja - Transformacja Hougha

### Cel:
- zapoznanie z transformacją Hougha dla pojedynczego punktu,
- kilku punktów, prostych figur
- wykorzystanie transformacji Hougha do detekcji linii prostych na rzeczywistym obrazie
- transformacja Hougha w przestrzeni ab

### Transformacja Hough'a

Transformacja Hougha dla prostych jest metodą detekcji współliniowych punktów. Każda prosta może być jednoznacznie przedstawiona za pomocą dwóch parametrów. Przestrzeń tych parametrów to przestrzeń Hougha. Najczęściej wykorzystywanymi parametrami w tej metodzie są współczynniki ρ,θ

opisujące równanie prostej w postaci normalnej:

ρ=x⋅cos(θ)+y⋅sin(θ)

gdzie: ρ - promień wodzący, θ - kąt pomiędzy ρ a osią OX.

Własności transformacji Hougha:
- prostej w przestrzeni kartezjańskiej odpowiada punkt w przestrzeni Hougha
- pękowi prostych przechdzących przez punkt w przestrzeni kartezjańskiej odpowiada krzywa sinusoidalna w przestrzeni Hougha
- punkty leżące na tej samej prostej (w przestrzeni kartezjańskiej) korespondują z sinusoidami przechodzącymi przez wspólny punkt w przestrzeni Hougha.

Metoda wyliczania transformacji Hougha składa się z następujących kroków:
- przez każdy badany (różny od zera) punkt obrazu prowadzony jest pęk prostych, przechodzących przez ten punkt
- każda z tych prostych transformowana jest do przestrzeni Hougha i tworzy tam punkt o współrzędnych ρ,θ
- w ten sposób, każdy punkt obrazu pierwotnego (pęk prostych) jest odwzorowany w sinusoidalną krzywą w przestrzeni Hougha

Przestrzeń Hougha jest przestrzenią akumulacyjną tzn. punkty sinusoidalnych krzywych, wygenerowanych dla punktów obrazu pierwotnego dodają się w miejscach, w których krzywe te przecinają się. Powstałe w ten sposób (w przestrzeni Hougha) maksima odpowiadają zbiorom punktów, należących do jednej prostej. Współrzędne ρ,θ
tego maksimum jednoznacznie określają położenie prostej na obrazie pierwotnym.

### Transformacja Hougha dla małej liczby punktów.
   1. Uruchom poniższy kod. W tablicy `im` wskaż jeden punkt, dla którego ma zostać obliczona transformata.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
import os
import requests

In [ ]:
url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/'

fileNames = ["dom.png", "kwadraty.png", "lab112.png"]
for fileName in fileNames:
  if not os.path.exists(fileName):
      r = requests.get(url + fileName, allow_redirects=True)
      open(fileName, 'wb').write(r.content)

image0 = np.zeros((64,64), dtype=np.uint8)

image0[18, 31] = 1

fig, ax = plt.subplots()
fig.set_size_inches(4, 4)
ax.imshow(image0, 'gray')
ax.axis('off')


3. Wykonaj transformację Hougha obazu im. Wykorzystaj funkcję *hough_line* z modułu _skimage.transform_. Funkcja zwraca: macierz H (przestrzeń Hougha) oraz dwa wektory theta i rho dla kolejnych 
4. Wyświetl przestrzeń Hougha za pomocą funkcji _plt.imshow_ (można też wykorzystać poniższą funkcję *show_hough*). Jak "wygląda" pojedynczy punkt w przestrzeni Hougha?

In [ ]:
def show_hough(h, image, title=""):
    # Generating figure 1
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    ax = axes.ravel()

    ax[0].imshow(image, 'gray')
    ax[0].set_title('Input image: ' + title)
    ax[0].set_axis_off()

    ax[1].imshow(h, 'gray')
    ax[1].set_title('Hough transform')
    ax[1].set_xlabel('Angles (degrees)')
    ax[1].set_ylabel('Distance (pixels)')
    ax[1].axis('image')
    
    plt.tight_layout()
    plt.show()    

5. Powtórz punkty 1-4, ale tym razem klinkij dwa punkty. Jak zmienia się przestrzeń Hougha?
6. Powtórz punkty 1-4, ale tym razem kliknij kilka punktów starając się aby były współliniowe. Zaobserwuj zmiany w przestrzeni Hougha
7. Poeksperymentuj z różnymi układami punktów

In [ ]:
hough = hough_line(image0)
image0[32, 32] = 1
show_hough(hough[0], image=image0, title="2 point")

image1 = np.zeros((64,64), dtype=np.uint8)
image1[32, 32] = 1
image1[20, 20] = 1
image1[44, 10] = 1
image1[10, 44] = 1
hough = hough_line(image1)
show_hough(hough[0], image1, title="4 points")

image2 = np.zeros((64,64), dtype=np.uint8)
#make circle
for i in range(64):
  for j in range(64):
    if (i-32)**2 + (j-32)**2 < 20**2:
      image2[i, j] = 1
hough = hough_line(image2)
show_hough(hough[0], image2, title="circle")

image3 = np.zeros((64,64), dtype=np.uint8)
#make a cross
for i in range(64):
  for j in range(64):
    if abs(i-32) < 5 or abs(j-32) < 5:
      image3[i, j] = 1
hough = hough_line(image3)
show_hough(hough[0], image3, title="cross")

### Transformata Hougha dla pojedynczego obiektu

W tym podpunkcie pokazane zostanie praktycznie wykorzystanie transformaty Hougha - do detekcji prostych na sztucznym rysunku.

   1. Wczytaj obraz "kwadraty.png". Wyświetl go.
   2. Wykonaj detekcję krawędzi jedną z metod gradientowych. Ważne aby obraz krawędzi był jak najlepszej jakości - co oznacza cienkie (nawet niekoniecznie ciągłe) krawędzie - dla tego przypadku nie powinno być trudne do uzyskania. Wyświetl obraz po detekcji krawędzi.
   3. Wykonaj transformatę Hougha obrazu krawędziowego. Wykorzystaj funkcję *hough\_line*.
   4. Wyświetl macierz H. Czy widoczna jest taka liczba maksimów jakiej się spodziewamy?

In [ ]:
img = cv2.imread('kwadraty.png',cv2.IMREAD_GRAYSCALE)
img_canny=cv2.Canny(img,5,10,None,3,1)

hough_square = hough_line(img_canny)
show_hough(hough_square[0], img_canny, title="squares")

 5. W module skimage.transform dostępna jest funkcja do automatycznej analizy przestrzeni Hougha - wyszukiwania maksimów - *hough\_line\_peaks*. Jako parametry przyjmuje ona wyniki funkcji *hough\_line* (macierz H, theta i rho). Dodatkowo można podać próg powyżej którego punkt uznawany jest za maksimum (_threshold_ - domyslnie jest to połowa maksimum w przestrzeni H) oraz liczbę poszukiwanych maksimów (*num_peaks*). Funkcja zwraca współrzędne maksimów. Wykorzystaj funkcję *hough\_line\_peaks* do znalezienia maksimów odpowiadających krawędziom kwadratów.
 6. Wyświetl macierz H używając konstrukcji:

In [ ]:
h_matrix,theta,rho = hough_line(img_canny)
hspace, angles, dists = hough_line_peaks(h_matrix,theta,rho)
H = hough_line_peaks(h_matrix,theta,rho)

fig,ax = plt.subplots(1)
ax.set_aspect('equal')

ax.imshow(H, 'gray') # gdzie H jest przestrzenią Hougha

Taki zapis pozwoli na dołożenie annotacji (okręgów) w miejscach znalezionych maksimów. Wyrysowanie okręgu w punkcie x, y (o rozmiarze 10, w czerwonym kolorze, bez wypełnienia środka) realizuje wywołanie: 

**circle = plt.Circle((x, y), 10, color='r', fill=False)**

natomiast dołożenie takiego okręgu do obrazu to:

**ax.add_patch(circle)**

Zaznacz maksima na obrazie wykorzystując rezultat funkcji *hough\_line\_peaks* biorąc pod uwagę, że zwraca ona kąty w radianach z przedziału od -pi/2 do pi/2, a rho z przedziału od -r/2 do r/2 gdzie r to pionowy rozmiar przestrzeni Hougha. 

In [ ]:
f, ax = plt.subplots(1, figsize=(20, 15))
plt.imshow(h_matrix, 'gray')
plt.axis('off')
height, width = h_matrix.shape

for angle, dist in zip(angles, dists):
    x, y = (angle / np.pi + 0.5) * width, int(dist + height / 2)

    circle = plt.Circle((x, y), 10, color='g', fill=False)
    ax.add_patch(circle)

plt.show()



7. Istnieje też możliwość przeprowadzenia transformacji Hougha z użyciem biblioteki OpenCV. W bibliotece znajdują się dwie wersje funkcji wyszukującej linie proste - 'klasyczna' - _HoughLines_ oraz probabilistyczna _HoughLinesP_. Zadna z nich nie zwraca przestrzeni Hougha. Wynikiem działania pierwszej jest lista parametrów prostych (krotki zawierające rho, theta). Druga zwraca krotki 4-ro elementowe ze współrzędnymi końców odcinków wykorzystanych do wylicznia parametrów (czyli znalezienia prostej). 
8. Wyznacz linie obecne na obrazie za pomocą funkcji _HoughLines_. Wykryte linie wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). Do wyświetlania linii wykorzystaj przykładowy kod:

In [ ]:
img = cv2.imread('kwadraty.png')
img_canny = cv2.Canny(cv2.imread('kwadraty.png', cv2.IMREAD_GRAYSCALE), 1, 255, None, 3, 1)

lines = cv2.HoughLines(img_canny,1,np.pi / 180,45)
for rho, theta in lines[:, 0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))  
    plt.plot((x1, x2), (y1, y2), 'r-')
      
plt.imshow(img)
plt.axis('off')
plt.show()


9. Wyznacz odcinki obecne na obrazie za pomocą funkcji _HoughLinesP_. Wykryte odcinki wyrysuj na obrazie początkowym (UWAGA: wczytanym bez konwersji na graylevel). 

In [ ]:
img = cv2.imread(filename='kwadraty.png')
img_detected = cv2.Canny(cv2.imread(filename='kwadraty.png',flags=cv2.IMREAD_GRAYSCALE), 1, 255, None, 3, 1)
lines = cv2.HoughLinesP(img_detected, 1, np.pi/180, 40, minLineLength=20, maxLineGap=50)

if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        plt.plot((x1, x2), (y1, y2), 'r-')

plt.imshow(img)
plt.axis('off')
plt.show()

### Transformata Hougha dla obrazu rzeczywistego.

Bazując na kodzie stworzonym w punkcie B wyszukamy linie na obrazie rzeczywistym.
   1. Wczytaj obraz "lab112.png". Wyświetl go.
   2. Wykorzystując wszystkie poznane techniki przetwarzania obrazów (filtracja, przekształcenia morfologiczne, binaryzację, detekcję krawędzi) wyodrębnij krawędzie samych kwadratów - tak aby były jak najlepszej jakości (cienkie) - jednocześnie eliminując z obrazu zakłócenia.
   3. Wykorzystaj funkcje *hough_line* i *hough_line_peaks* do detekcji linii na obrazie, a następnie np. wykorzystując kod z punktu 8 poprzedniego ustępu wyrysuj na oryginalnym obrazie znalezione linie.

In [ ]:
img = cv2.imread('lab112.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(img,'gray')

In [ ]:
img_filtered = cv2.GaussianBlur(img, (5, 5), 0)
edges = cv2.Canny(image=img, threshold1=160, threshold2=240)
plt.imshow(edges,'gray')

In [ ]:

lines = cv2.HoughLinesP(image=edges, rho=1, theta=np.pi/180, threshold=50, minLineLength=90, maxLineGap=10)

for line in lines:
    x1, y1, x2, y2 = line[0]
    plt.plot((x1, x2), (y1, y2), 'r-')

plt.imshow(img, 'gray')

4. Wczytaj obraz "dom.png". Wypróbuj działanie transformacji Hougha na tym obrazie z wykorzystaniem funkcji _cv2.HoughLinesP_  (oczywiście po odpowiednich przekształceniach). Postaraj się tak przygotować obraz z krawędziami i dobrać parametry aby wyrysować na oryginalnym obrazie odcinki obejmujące zarysy domu. Weź pod uwage dodatkowe parametry funkcji, takie jak:   minLineLength, maxLineGap.

In [ ]:
edges = cv2.Canny(cv2.GaussianBlur(cv2.imread("dom.png", cv2.IMREAD_GRAYSCALE), (3, 3), 0), 100, 200, None, 3, 1)
plt.imshow(edges,'gray')

In [ ]:
img = cv2.imread('dom.png')
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 40, minLineLength=20, maxLineGap=10)
for line in lines:
    x1, y1, x2, y2 = line[0]
    plt.plot((x1, x2), (y1, y2), 'b-')

plt.imshow(img, 'gray')